In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('train.csv', index_col=0)
df.head()

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
7,0,2,2,5,1,0,0,1,0,0,...,9,1,5,8,0,1,1,0,0,1
9,0,1,1,7,0,0,0,0,1,0,...,3,1,1,9,0,1,1,0,1,0
13,0,5,4,9,1,0,0,0,1,0,...,4,2,7,7,0,1,1,0,1,0
16,0,0,1,2,0,0,1,0,0,0,...,2,2,4,9,0,0,0,0,0,0
17,0,0,2,0,1,0,1,0,0,0,...,3,1,1,3,0,0,0,1,1,0


In [33]:
X = df.drop(['target'], axis=1).values
y = df.target.values
X.shape, y.shape

((595212, 57), (595212,))

In [38]:
X_1 = X[y == 1]
y_1 = y[y == 1]
X_0 = X[y == 0][:X_1.shape[0], :]
y_0 = y[y == 0][:X_1.shape[0]]
X_0.shape, X_1.shape, y_1.shape, y_0.shape

((21694, 57), (21694, 57), (21694,), (21694,))

In [48]:
X = np.concatenate((X_0, X_1), axis=0)
y = np.concatenate((y_0, y_1), axis=0)
X.shape, y.shape

((43388, 57), (43388,))

In [49]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X = ss.fit_transform(X)

### 80-20 Split

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

D = 57
X_train_0 = X_train[y_train == 0]
X_train_1 = X_train[y_train == 1]
X_test_0 = X_test[y_test == 0]
X_test_1 = X_test[y_test == 1]

M_0 = X_train_0.mean(axis=0)
M_1 = X_train_1.mean(axis=0)

cov_0 = np.zeros((D, D))
for i in range(X_train_0.shape[0]):
    W = (X_train_0[i] - M_0).reshape(-1, 1)
    cov_0 += np.matmul(W, W.T)

cov_1 = np.zeros((D, D))
for i in range(X_train_1.shape[0]):
    W = (X_train_1[i] - M_1).reshape(-1, 1)
    cov_1 += np.matmul(W, W.T)

S_w = cov_0 + cov_1
S_w_inv = np.linalg.pinv(S_w)
W = np.matmul(S_w_inv, (M_1 - M_0)).reshape(-1, 1)
W = W/np.linalg.norm(W,2)

def predict(x, W, b):
    return np.matmul(W.T, x) + b


# Line Search
best_b, best_acc = 0, 0
lower = np.matmul(W.T, M_0)[0] - 5
upper = np.matmul(W.T, M_1)[0] + 5
for b in np.linspace(lower, upper, 1000):
    acc = 0
    for (i, X_i) in enumerate(X_train_0):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred < 0:
            acc += 1

    for (i, X_i) in enumerate(X_train_1):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred >= 0:
            acc += 1

    if acc > best_acc:
        best_acc = acc
        best_b = b
    
    # print(f'Bias {b}, Acc: {acc/X_train.shape[0]}, Best: {best_acc/X_train.shape[0]}')

print('Train acc. =', best_acc/X_train.shape[0])

b = best_b
acc = 0
for (i, X_i) in enumerate(X_test_0):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        acc += 1

for (i, X_i) in enumerate(X_test_1):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred >= 0:
        acc += 1

print(f'Test acc. = {acc/X_test.shape[0]}')

# f1-score
y_pred = []
for X_i in X_train:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)
print('F1-Score (train) = ', f1_score(y_train, y_pred))

y_pred = []
for X_i in X_test:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)

print('F1-Score (test) = ', f1_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Train acc. = 0.5914433880726016
Test acc. = 0.5926480755934547
F1-Score (train) =  0.5751774961804619
F1-Score (test) =  0.5729128911441345


array([[2772, 1587],
       [1948, 2371]])

In [56]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_train)
print(svc.score(X_train, y_train))
print('F1-Score (train) = ', f1_score(y_train, y_pred))
y_pred = svc.predict(X_test)
print(svc.score(X_test, y_test))
print('F1-Score (test) = ', f1_score(y_test, y_pred))

0.708038029386344
F1-Score (train) =  0.7018359420972107
0.5852731044019359
F1-Score (test) =  0.5757397147235648


In [57]:
from sklearn.neural_network import MLPClassifier

svc = MLPClassifier()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_train)
print(svc.score(X_train, y_train))
print('F1-Score (train) = ', f1_score(y_train, y_pred))
y_pred = svc.predict(X_test)
print(svc.score(X_test, y_test))
print('F1-Score (test) = ', f1_score(y_test, y_pred))

0.732987611639297
F1-Score (train) =  0.7362399681256759
0.5562341553353307
F1-Score (test) =  0.5586246418338109


/home/svp/miniconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### 70-30 Split

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

D = 57
X_train_0 = X_train[y_train == 0]
X_train_1 = X_train[y_train == 1]
X_test_0 = X_test[y_test == 0]
X_test_1 = X_test[y_test == 1]

M_0 = X_train_0.mean(axis=0)
M_1 = X_train_1.mean(axis=0)

cov_0 = np.zeros((D, D))
for i in range(X_train_0.shape[0]):
    W = (X_train_0[i] - M_0).reshape(-1, 1)
    cov_0 += np.matmul(W, W.T)

cov_1 = np.zeros((D, D))
for i in range(X_train_1.shape[0]):
    W = (X_train_1[i] - M_1).reshape(-1, 1)
    cov_1 += np.matmul(W, W.T)

S_w = cov_0 + cov_1
S_w_inv = np.linalg.inv(S_w)
W = np.matmul(S_w_inv, (M_1 - M_0)).reshape(-1, 1)
W = W/np.linalg.norm(W,2)

def predict(x, W, b):
    return np.matmul(W.T, x) + b


# Line Search
best_b, best_acc = 0, 0
lower = np.matmul(W.T, M_0)[0] - 5
upper = np.matmul(W.T, M_1)[0] + 5
for b in np.linspace(lower, upper, 1000):
    acc = 0
    for (i, X_i) in enumerate(X_train_0):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred < 0:
            acc += 1

    for (i, X_i) in enumerate(X_train_1):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred >= 0:
            acc += 1

    if acc > best_acc:
        best_acc = acc
        best_b = b
    
    # print(f'Bias {b}, Acc: {acc/X_train.shape[0]}, Best: {best_acc/X_train.shape[0]}')

print('Train acc. =', best_acc/X_train.shape[0])

b = best_b
acc = 0
for (i, X_i) in enumerate(X_test_0):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        acc += 1

for (i, X_i) in enumerate(X_test_1):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred >= 0:
        acc += 1

print(f'Test acc. = {acc/X_test.shape[0]}')

# f1-score
y_pred = []
for X_i in X_train:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)
print('F1-Score (train) = ', f1_score(y_train, y_pred))

y_pred = []
for X_i in X_test:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)

print('F1-Score (test) = ', f1_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Train acc. = 0.5922425998485398
Test acc. = 0.5866175001920565
F1-Score (train) =  0.5746084089035449
F1-Score (test) =  0.5702419934510022


array([[4066, 2416],
       [2965, 3570]])

In [53]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_train)
print(svc.score(X_train, y_train))
print('F1-Score (train) = ', f1_score(y_train, y_pred))
y_pred = svc.predict(X_test)
print(svc.score(X_test, y_test))
print('F1-Score (test) = ', f1_score(y_test, y_pred))

0.7146290869579534
F1-Score (train) =  0.7033373267157282
0.5844664669278635
F1-Score (test) =  0.5719711956951808


In [54]:
from sklearn.neural_network import MLPClassifier

svc = MLPClassifier()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_train)
print(svc.score(X_train, y_train))
print('F1-Score (train) = ', f1_score(y_train, y_pred))
y_pred = svc.predict(X_test)
print(svc.score(X_test, y_test))
print('F1-Score (test) = ', f1_score(y_test, y_pred))

0.7440650620657865
F1-Score (train) =  0.743185647735157
0.5525082584312822
F1-Score (test) =  0.5545614437562132


/home/svp/miniconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
